### Reduce item_type_code_name levels

In [48]:
%%capture

%run '../lib/libraries.ipynb'

In [49]:
filterDf  = load_df('filterDf')
types = get_var_types()

###### Display levels

In [50]:
count_values(filterDf, 'item_type_code_name').show(50, False)

+----------------------+-----+
|item_type_code_name   |count|
+----------------------+-----+
|multipleChoice        |26704|
|fillInTheBlank        |10497|
|equationEntry         |7841 |
|trueFalse             |4561 |
|cloze                 |3024 |
|multipleSelect        |2295 |
|graphing              |2076 |
|MultipleChoiceResponse|1550 |
|choiceMatrix          |1203 |
|matching              |924  |
|shortAnswer           |853  |
|selectText            |813  |
|bucketing             |799  |
|sortable              |549  |
|essay                 |531  |
|numberLine            |302  |
|aheAlgo               |219  |
|imageLabel            |80   |
|RubricResponse        |11   |
|FillinBlankResponse   |5    |
|fileUpload            |1    |
+----------------------+-----+



###### Display item_type_code_name and scoring_type_code Corelation

In [51]:
dfPd = filterDf.select('item_type_code_name','scoring_type_code').toPandas()
pd.crosstab(dfPd.item_type_code_name.fillna('null'), dfPd.scoring_type_code.fillna('null'), margins=True, margins_name="Total")

scoring_type_code,[unassigned],automatic,external,manual,Total
item_type_code_name,,,,,
FillinBlankResponse,5,0,0,0,5
MultipleChoiceResponse,1550,0,0,0,1550
RubricResponse,11,0,0,0,11
aheAlgo,0,0,219,0,219
bucketing,0,799,0,0,799
choiceMatrix,0,1203,0,0,1203
cloze,0,3024,0,0,3024
equationEntry,0,7841,0,0,7841
essay,9,0,0,522,531


- unassigned
    - FillinBlankResponse
    - MultipleChoiceResponse
    - shortAnswer
- manual
  - aheAlg

In [52]:
encode_categories_as_swoe(filterDf).select('item_type_code_name', 'item_type_code_name_swoe').show()

+-------------------+------------------------+
|item_type_code_name|item_type_code_name_swoe|
+-------------------+------------------------+
|         fileUpload|       0.958169577610213|
|FillinBlankResponse|      0.6523843277995943|
|FillinBlankResponse|      0.6523843277995943|
|FillinBlankResponse|      0.6523843277995943|
|FillinBlankResponse|      0.6523843277995943|
|FillinBlankResponse|      0.6523843277995943|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equationEntry|      0.6998692812113955|
|      equati